In [14]:
import gym
import gym_pen
import numpy as np
import tensorflow as tf

shape = (500, 500)

env = gym.make('pen-v0')

observation, _ = env.reset()

def custom_model(shape=(100, 100)):
    # Define the layers
    array_input = tf.keras.layers.Input(shape=(shape[0], shape[1], 1))
    char_input = tf.keras.layers.Input(shape=(1,))
    index_input = tf.keras.layers.Input(shape=(1,))
    xy_input = tf.keras.layers.Input(shape=(2,))

    array_x = tf.keras.layers.Conv2D(15, 3, activation='relu', kernel_initializer='he_normal')(array_input)
    array_x = tf.keras.layers.MaxPooling2D(5)(array_x)
    array_x = tf.keras.layers.Conv2D(15, 3, activation='relu', kernel_initializer='he_normal')(array_x)
    array_x = tf.keras.layers.Flatten()(array_x)

    combined_x = tf.keras.layers.Concatenate()([array_x, index_input, xy_input])
    x = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal')(combined_x)
    x = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal')(x)

    output_xy = tf.keras.layers.Dense(2, activation='relu')(x)

    model = tf.keras.Model(inputs=[array_input, index_input, xy_input], outputs=[output_xy])
    return model

model = custom_model(shape=shape)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_37 (InputLayer)       [(None, 500, 500, 1)]        0         []                            
                                                                                                  
 conv2d_18 (Conv2D)          (None, 498, 498, 15)         150       ['input_37[0][0]']            
                                                                                                  
 max_pooling2d_9 (MaxPoolin  (None, 99, 99, 15)           0         ['conv2d_18[0][0]']           
 g2D)                                                                                             
                                                                                                  
 conv2d_19 (Conv2D)          (None, 97, 97, 15)           2040      ['max_pooling2d_9[0][0]'

In [17]:
env.initialize_with_noise(0.01)

In [18]:

# 모델 생성
model = custom_model()

# 모델 초기화
model.build(input_shape=[(None, 500, 500, 1), (None, 1), (None, 2)])

# 가상의 입력 데이터 생성 (with batch dimension)
array_input_data = np.expand_dims(env.model_canvas, axis=0)  # (1, 100, 100, 1)
index_input_data = np.array([[env.step_idx]])  # (1, 1)
xy_input_data = np.array([env.model_pen_position])  # (1, 2)


# 모델에 입력 데이터 전달하여 출력 얻기
output_xy = model.predict([array_input_data, index_input_data, xy_input_data])

np.shape(output_xy[0])
print(output_xy[0,:])

1/1 [==============================] - 0s 52ms/step
[0.00062219 0.0086446 ]
